In [1]:
import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv("0304.csv", index_col=0, parse_dates=['timestamp']).fillna(0)
initial = df['timestamp'].iloc[0].value
df['timestamp'] = df['timestamp'].apply(lambda x: int((x.value - initial) / 10e10) / 3)

In [23]:
df

,Unnamed: 0.1,timestamp,station_id,lane_type,obs_percentage,total_flow,avg_occupancy,avg_speed,lane_0_flow,lane_0_avg_occ,...,lane_6_observed,lane_7_flow,lane_7_avg_occ,lane_7_avg_speed,lane_7_observed,abs_pm,latitude,longitude,lanes,name
124416,0,0.0,715972,OR,100,1.0,0.0010,0.0,1.0,0.0010,...,0,0.0,0.0,0.0,0,40.989,34.120764,-117.888430,1,CITRUS NB
124417,1,1.0,715972,OR,100,2.0,0.0024,0.0,2.0,0.0024,...,0,0.0,0.0,0.0,0,40.989,34.120764,-117.888430,1,CITRUS NB
124418,2,2.0,715972,OR,100,0.0,0.0000,0.0,0.0,0.0000,...,0,0.0,0.0,0.0,0,40.989,34.120764,-117.888430,1,CITRUS NB
124419,3,3.0,715972,OR,100,3.0,0.0028,0.0,3.0,0.0028,...,0,0.0,0.0,0.0,0,40.989,34.120764,-117.888430,1,CITRUS NB
124420,4,4.0,715972,OR,100,5.0,0.0046,0.0,5.0,0.0046,...,0,0.0,0.0,0.0,0,40.989,34.120764,-117.888430,1,CITRUS NB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165883,41467,283.0,775859,FR,100,12.0,0.0044,0.0,8.0,0.0057,...,0,0.0,0.0,0.0,0,39.159,34.127006,-117.917217,2,VERNON
165884,41468,284.0,775859,FR,100,5.0,0.0021,0.0,5.0,0.0042,...,0,0.0,0.0,0.0,0,39.159,34.127006,-117.917217,2,VERNON
165885,41469,285.0,775859,FR,100,2.0,0.0006,0.0,2.0,0.0013,...,0,0.0,0.0,0.0,0,39.159,34.127006,-117.917217,2,VERNON
165886,41470,286.0,775859,FR,100,2.0,0.0006,0.0,2.0,0.0012,...,0,0.0,0.0,0.0,0,39.159,34.127006,-117.917217,2,VERNON


In [2]:
a = [1,2,3,3,4]
print(a)
print(list(set(a)))

[1, 2, 3, 3, 4]
[1, 2, 3, 4]


In [24]:
stations = pd.DataFrame(df.groupby(by="station_id", as_index=False)).set_index(0)

In [37]:
stations = pd.DataFrame(df.groupby(by="station_id")).set_index(0)
fr_pm = {}
stations.iloc[0,0]['abs_pm'].iloc[0]
for i in range(len(stations)):
    if stations.iloc[i,0]['lane_type'].iloc[0] == "FR":
        abs_pm = stations.iloc[i,0]['abs_pm'].iloc[0]
        fr_pm[stations.iloc[i,0]['station_id'].iloc[0]] = abs_pm
# print(fr_pm)
# print(len(fr_pm))
station_pm = {}
for i in range(len(stations)):
    abs_pm = stations.iloc[i,0]['abs_pm'].iloc[0]
    station_pm[stations.iloc[i,0]['station_id'].iloc[0]] = abs_pm
# print(station_pm)
fr_nearest_station = {}
for fr in fr_pm:
    nearest_station = max([(station_id, abs_pm) for station_id, abs_pm  in station_pm.items() if (abs_pm < fr_pm[fr] and stations[1].loc[station_id]['lane_type'].iloc[0] != "OR" and stations[1].loc[station_id]['lane_type'].iloc[0] != "FR")], key=lambda x: x[1])
    fr_nearest_station[fr] = nearest_station[0]
print(fr_nearest_station)
for fr in fr_nearest_station:
    if stations[1].loc[fr_nearest_station[fr]]['lane_type'].iloc[0] == "OR" or stations[1].loc[fr_nearest_station[fr]]['lane_type'].iloc[0] == "FR":
        print("[Error: No Main Line Found Before Off-Ramp]")
        exit()

{717651: 717646, 717658: 717654, 717666: 773154, 717671: 717667, 717680: 761214, 717691: 772917, 737480: 717640, 761130: 773193, 761154: 761138, 761167: 761149, 761228: 772903, 763908: 717640, 767650: 716026, 767912: 767895, 768923: 717646, 769139: 771603, 769269: 717633, 769272: 717635, 771605: 772966, 774990: 772872, 775859: 772887}


In [39]:
fr_split_ratios = {}
k=0
for fr, ml in fr_nearest_station.items():
    k+=1
#     print(df[df['station_id']==fr]['total_flow'])
#     print(df[df['station_id']==ml]['total_flow'])
#     print(df[df['station_id']==fr]['total_flow'].divide(df[df['station_id']==ml]['total_flow'].values))
#     print(df[df['station_id']==ml])
    temp = pd.Series([0] * 48, index=list(range(0,48)), name='split_ratio')
    # update on half-hour basis
    for i in range(0, 48):
        if sum(df[df['station_id']==ml]['total_flow'].values[i:i+6]) == 0:
            temp.loc[i] = 0
            continue
        temp.loc[i] = sum(df[df['station_id']==fr]['total_flow'].values[i:i+6]) / sum(df[df['station_id']==ml]['total_flow'].values[i:i+6])
    fr_split_ratios[fr] = temp
fr_split_ratios

{717651: 0     0.019549
 1     0.025335
 2     0.026194
 3     0.020313
 4     0.020767
 5     0.024306
 6     0.020408
 7     0.009804
 8     0.013834
 9     0.014228
 10    0.019780
 11    0.018059
 12    0.021322
 13    0.026846
 14    0.031250
 15    0.027295
 16    0.034568
 17    0.040609
 18    0.039773
 19    0.042169
 20    0.039157
 21    0.049231
 22    0.035948
 23    0.035599
 24    0.042208
 25    0.035948
 26    0.030928
 27    0.021127
 28    0.014235
 29    0.007519
 30    0.000000
 31    0.003774
 32    0.003717
 33    0.007246
 34    0.007143
 35    0.021429
 36    0.020979
 37    0.017361
 38    0.016181
 39    0.019108
 40    0.017857
 41    0.013812
 42    0.013333
 43    0.012531
 44    0.012788
 45    0.012255
 46    0.012107
 47    0.004515
 Name: split_ratio, dtype: float64,
 717658: 0     0.050290
 1     0.045455
 2     0.064516
 3     0.064965
 4     0.071605
 5     0.076705
 6     0.069697
 7     0.079755
 8     0.072785
 9     0.069767
 10    0.062731
 11 

In [41]:
sum(stations.apply(lambda x: len(x)==1, axis=1))

144

In [48]:
sum(stations==288) == len(stations)

True

In [15]:
df[df['abs_pm'] == min(df['abs_pm'])]['station_id'].iloc[0]

717631

In [29]:
initial = df['timestamp'].iloc[0].value
df['timestamp'] = df['timestamp'].apply(lambda x: ( (x.value - initial) / 10e10) / 3)

In [32]:
df.sort_values('timestamp')

,Unnamed: 0.1,timestamp,station_id,lane_type,obs_percentage,total_flow,avg_occupancy,avg_speed,lane_0_flow,lane_0_avg_occ,...,lane_6_observed,lane_7_flow,lane_7_avg_occ,lane_7_avg_speed,lane_7_observed,abs_pm,latitude,longitude,lanes,name
124416,0,0.0,715972,OR,100,1.0,0.0010,0.0,1.0,0.0010,...,0,0.0,0.0,0.0,0,40.989,34.120764,-117.888430,1,CITRUS NB
131904,7488,0.0,717658,FR,100,4.0,0.0035,0.0,4.0,0.0070,...,0,0.0,0.0,0.0,0,30.299,34.148554,-118.064055,2,MICHILLINDA
158400,33984,0.0,771605,FR,100,6.0,0.0061,0.0,6.0,0.0061,...,0,0.0,0.0,0.0,0,45.949,34.119720,-117.803812,1,SAN DIMAS AVE
131616,7200,0.0,717654,ML,100,75.0,0.0168,66.6,2.0,0.0015,...,0,0.0,0.0,0.0,0,30.029,34.148522,-118.068683,4,ROSEMEAD 1
158688,34272,0.0,771606,HV,100,10.0,0.0065,65.0,10.0,0.0065,...,0,0.0,0.0,0.0,0,45.949,34.119720,-117.803812,1,SAN DIMAS AVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151487,27071,287.0,767896,ML,100,103.0,0.0304,68.0,38.0,0.0237,...,0,0.0,0.0,0.0,0,47.789,34.119372,-117.771695,3,BIXBY DR
151199,26783,287.0,767895,HV,100,10.0,0.0068,64.9,10.0,0.0068,...,0,0.0,0.0,0.0,0,47.789,34.119372,-117.771695,1,BIXBY DR
150911,26495,287.0,767884,OR,100,1.0,0.0009,0.0,1.0,0.0009,...,0,0.0,0.0,0.0,0,47.289,34.119703,-117.780403,1,FOOTHILL BL
152639,28223,287.0,767926,OR,100,0.0,0.0000,0.0,0.0,0.0000,...,0,0.0,0.0,0.0,0,48.489,34.119291,-117.759529,1,FRUIT ST
